In [ ]:
import os
import pandas as pd
import tensorflow as tf
import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.layers import Dense, Activation, Flatten, Conv2D, Lambda, MaxPooling2D, Dropout, Input
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import keras.backend as K
import pickle
from tensorflow.keras import layers
import random
import h5py
import pandas as pd
import tensorflow.keras.backend as K
import tensorflow_hub

In [ ]:
# HYPERPARAMETERS
BUCKETS = 5
MAX_ANGLE = 450 # May need to confirm this
one_hot_length = int(900/BUCKETS)
BINS = np.arange(0, MAX_ANGLE - (MAX_ANGLE % BUCKETS) + 2*BUCKETS, BUCKETS)
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 64 # 128, 64, 32, 8, 6
DATA_VALUE = 'X'
Y_VALUE_1 = 'speed'
Y_VALUE_2 = 'steering_angle'
timesteps = 100 # 64, 5
interval = 10
shift_amount = 10
number_outputs = int(timesteps/interval)
img_shape = (128, 128, 3) #(100, 100, 3) # (3, 160, 320), # 
predicting_frame_num = 3
error_margin = 1

# GPU

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

# Data Paths

In [ ]:
train_files = ['/content/drive/Shareddrives/ELEC 494 - Ω2Ω/Data/camera/2016-06-08--11-46-01.h5', # 2.64
               '/content/drive/Shareddrives/ELEC 494 - Ω2Ω/Data/camera/2016-01-30--11-24-51.h5', # 7.62
               '/content/drive/Shareddrives/ELEC 494 - Ω2Ω/Data/camera/2016-01-30--13-46-00.h5', # 8.5
               '/content/drive/Shareddrives/ELEC 494 - Ω2Ω/Data/camera/2016-02-02--10-16-58.h5', # 8.06
               '/content/drive/Shareddrives/ELEC 494 - Ω2Ω/Data/camera/2016-03-29--10-50-20.h5',  # 11.28 GB
               '/content/drive/Shareddrives/ELEC 494 - Ω2Ω/Data/camera/2016-02-11--21-32-47.h5'] # 12.3 GB

train_labels = ['/content/drive/Shareddrives/ELEC 494 - Ω2Ω/Data/log/2016-06-08--11-46-01_NEW.h5',
                '/content/drive/Shareddrives/ELEC 494 - Ω2Ω/Data/log/2016-01-30--11-24-51_NEW.h5',
                '/content/drive/Shareddrives/ELEC 494 - Ω2Ω/Data/log/2016-01-30--13-46-00_NEW.h5', 
                '/content/drive/Shareddrives/ELEC 494 - Ω2Ω/Data/log/2016-02-02--10-16-58_NEW.h5', 
                '/content/drive/Shareddrives/ELEC 494 - Ω2Ω/Data/log/2016-03-29--10-50-20_NEW.h5', 
                '/content/drive/Shareddrives/ELEC 494 - Ω2Ω/Data/log/2016-02-11--21-32-47_NEW.h5']

train_small_imgs = train_files[0:2]
train_medium_imgs = train_files[0:4]
train_large_imgs = train_files[:]

train_tfRecord_path_s = list(map(lambda x: x.replace('camera', 'TFRecords/Data_2').replace('.h5', '.tfrecord'), train_small_imgs))
train_tfRecord_path_m = list(map(lambda x: x.replace('camera', 'TFRecords/Data_2').replace('.h5', '.tfrecord'), train_medium_imgs))
train_tfRecord_path_l = list(map(lambda x: x.replace('camera', 'TFRecords/Data_2').replace('.h5', '.tfrecord'), train_large_imgs))

train_new_files = glob.glob('/content/drive/Shareddrives/ELEC 494 - Ω2Ω/Data/comma2k19TF/train/*')
train_pick = int(len(train_new_files)*.45)
train_new_files = random.choices(train_new_files, k=train_pick)

size = 0
for item in train_new_files:
  size += os.path.getsize(item)
print(size/1e9)

32.03473518


In [ ]:
val_files = ['/content/drive/Shareddrives/ELEC 494 - Ω2Ω/Data/camera/2016-01-31--19-19-25.h5', # 2.93
             '/content/drive/Shareddrives/ELEC 494 - Ω2Ω/Data/camera/2016-05-12--22-20-00.h5', # 7.47 GB
             '/content/drive/Shareddrives/ELEC 494 - Ω2Ω/Data/camera/2016-02-08--14-56-28.h5'] # 3.81

val_labels = ['/content/drive/Shareddrives/ELEC 494 - Ω2Ω/Data/log/2016-01-31--19-19-25_NEW.h5',
              '/content/drive/Shareddrives/ELEC 494 - Ω2Ω/Data/log/2016-05-12--22-20-00_NEW.h5', 
              '/content/drive/Shareddrives/ELEC 494 - Ω2Ω/Data/log/2016-02-08--14-56-28_NEW.h5']

validation_path_imgs_s = val_files[0]
validation_path_imgs_m = val_files[0:2]
validation_path_imgs_l = val_files[:]

val_tfRecord_path_s = list(map(lambda x: x.replace('camera', 'TFRecords/Data_2').replace('.h5', '.tfrecord'), validation_path_imgs_s))
val_tfRecord_path_m = list(map(lambda x: x.replace('camera', 'TFRecords/Data_2').replace('.h5', '.tfrecord'), validation_path_imgs_m))
val_tfRecord_path_l = list(map(lambda x: x.replace('camera', 'TFRecords/Data_2').replace('.h5', '.tfrecord'), validation_path_imgs_l))

val_new_files = glob.glob('/content/drive/Shareddrives/ELEC 494 - Ω2Ω/Data/comma2k19TF/val/*')
val_pick = int(len(val_new_files)*.35)
val_new_files = random.choices(val_new_files, k=val_pick)

size = 0
for item in val_new_files:
  size += os.path.getsize(item)
print(size/1e9)

10.101766556


In [ ]:
test_files = ['/content/drive/Shareddrives/ELEC 494 - Ω2Ω/Data/camera/2016-04-21--14-48-08.h5', # 4.39
              '/content/drive/Shareddrives/ELEC 494 - Ω2Ω/Data/camera/2016-06-02--21-39-29.h5'] # 6.45

test_labels = ['/content/drive/Shareddrives/ELEC 494 - Ω2Ω/Data/log/2016-04-21--14-48-08_NEW.h5',
               '/content/drive/Shareddrives/ELEC 494 - Ω2Ω/Data/log/2016-06-02--21-39-29_NEW.h5']

testing_path_imgs_s = test_files[0]
testing_path_imgs_ml = test_files[:]

test_tfRecord_path_s = list(map(lambda x: x.replace('camera', 'TFRecords/Data_2').replace('.h5', '.tfrecord'), testing_path_imgs_s))
test_tfRecord_path_ml = list(map(lambda x: x.replace('camera', 'TFRecords/Data_2').replace('.h5', '.tfrecord'), testing_path_imgs_ml))

test_new_files = glob.glob('/content/drive/Shareddrives/ELEC 494 - Ω2Ω/Data/comma2k19TF/test/*')

# Get Data

## Previous generator

In [ ]:
class generator():

    def __init__(self, method):
      
      assert method in ['bucketize', 'normal', 'only_angle', 'bucketize_angle'], "Method should be the following: 'bucketize', 'normal', 'only_angle', 'bucketize_angle'"

      if method == 'bucketize':
        self.output = self.bucketize

      elif method == 'normal':
        self.output = self.normal
      else:
        self.output = self.only_angle

    @staticmethod
    def _parse_features(example_proto):

      features = {'X': tf.io.FixedLenFeature([], dtype=tf.string), 
                  'steering_angle': tf.io.FixedLenFeature([], dtype=tf.float32), 
                  'speed': tf.io.FixedLenFeature([], dtype=tf.float32),}

      parsed_features = tf.io.parse_single_example(example_proto, features)
      feat_dtypes = [tf.float32, tf.string, tf.int64]

      data_types = {"steering_angle": tf.float32, "speed": tf.float32, "X": tf.uint8,}
                    
      image = tf.io.decode_raw(parsed_features['X'], tf.uint8)
      image = tf.reshape(image, shape=img_shape)
      speed = parsed_features['speed']
      angle = parsed_features['steering_angle']

      return image, (angle, speed)

    @staticmethod
    def get_inteval(x, outputs):

      angle, speed = outputs
      return x.batch(timesteps).get_single_element()[:-1:interval], (angle.batch(timesteps).get_single_element()[-1], speed.batch(timesteps).get_single_element()[-1])

    def bucketize(self, img, angle, speed):

      if angle <= MAX_ANGLE:
        if angle < 0:
          pos_neg = 1
        else:
          pos_neg = 0
        degree_bucketized = np.zeros((90,))
        indx = sum(BINS<np.abs(angle))-1
        degree_bucketized[indx] = 1

        yield(img, {'sign': pos_neg, 'magnitude': degree_bucketized, 'speed': speed})

    def bucketize_angle(self, img, angle, speed):

      if angle <= MAX_ANGLE:
        angle_bucketized = (sum(BINS<np.abs(angle))-1) * np.round(angle/(np.abs(angle)+1e-8)) # 11 -> 1, 5 -> 0, -5 -> 0, -11 -> -1, 
        yield(img, angle_bucketized)

    def bucketize_angle_one_hot(self, img, angle, speed):
      
      if angle <= MAX_ANGLE:

        new_MAX_ANGLE = 2 * MAX_ANGLE
        BINS_2 = np.arange(0, new_MAX_ANGLE - (new_MAX_ANGLE % BUCKETS) + 2*BUCKETS, BUCKETS)
        vector_lenght = int(new_MAX_ANGLE/BUCKETS)
        degree_bucketized = np.zeros((vector_lenght,))

        pos_angle = angle + MAX_ANGLE # Convert all angles to positive
        idx = (sum(BINS_2<np.abs(pos_angle))-1)

        degree_bucketized[idx] = 1
        yield(img, degree_bucketized)

    def normal(self, img, angle, speed):

      if angle <= MAX_ANGLE:
        yield(img, {'angle': angle, 'speed': speed})

    def only_angle(self, img, angle, speed):

      if angle <= MAX_ANGLE:
        yield(img, angle) # /MAX_ANGLE

    def __call__(self, tfRecord):

      dataset = tf.data.TFRecordDataset(tfRecord)
      dataset = dataset.map(generator._parse_features, num_parallel_calls=tf.data.AUTOTUNE)
      dataset = dataset.skip(tf.random.uniform(shape=(), minval=0, maxval=shift_amount, dtype=tf.int64))
      dataset = dataset.window(timesteps, shift=shift_amount, drop_remainder=True)
      dataset = dataset.map(generator.get_inteval)

      for img, (angle, speed) in dataset:
        # if angle <= MAX_ANGLE:
        #   yield(img, angle) # /MAX_ANGLE
        if angle <= MAX_ANGLE:
          angle_bucketized = (sum(BINS<np.abs(angle))-1) * np.round(angle/(np.abs(angle)+1e-8)) # 11 -> 1, 5 -> 0, -5 -> 0, -11 -> -1, 
          yield(img, angle_bucketized)

## new generator

In [ ]:
class generator():

    def __init__(self, method):
      
      assert method in ['bucketize', 'normal', 'only_angle', 'bucketize_angle', 
                        'one_hot_bucket_angle', '1_hot_angle_wSpeed'], "Method should be the following: 'bucketize', 'normal', 'only_angle', 'bucketize_angle', 'one_hot_bucket_angle', '1_hot_angle_wSpeed'"

      # This idea doesnt work :(
      if method == 'bucketize':
        self.output = self.bucketize

      elif method == 'normal':
        self.output = self.normal
      else:
        self.output = self.only_angle

    @staticmethod
    def _parse_features(example_proto):

      features = {'X': tf.io.FixedLenFeature([], dtype=tf.string), 
                  'steering_angle': tf.io.FixedLenFeature([], dtype=tf.float32), 
                  'speed': tf.io.FixedLenFeature([], dtype=tf.float32),}

      parsed_features = tf.io.parse_single_example(example_proto, features)
      feat_dtypes = [tf.float32, tf.string, tf.int64]

      data_types = {"steering_angle": tf.float32, "speed": tf.float32, "X": tf.uint8,}
                    
      image = tf.io.decode_raw(parsed_features['X'], tf.uint8)
      image = tf.reshape(image, shape=img_shape)
      speed = parsed_features['speed']
      angle = parsed_features['steering_angle']

      return image, (angle, speed)

    @staticmethod
    def get_inteval(x, outputs):

      angle, speed = outputs
      return x.batch(timesteps).get_single_element()[:-1:interval], (angle.batch(timesteps).get_single_element()[-1], speed.batch(timesteps).get_single_element()[-1])

    def bucketize(self, img, angle, speed):

      if angle <= MAX_ANGLE:
        if angle < 0:
          pos_neg = 1
        else:
          pos_neg = 0
        degree_bucketized = np.zeros((90,))
        indx = sum(BINS<np.abs(angle))-1
        degree_bucketized[indx] = 1

        yield(img, {'sign': pos_neg, 'magnitude': degree_bucketized, 'speed': speed})

    def bucketize_angle(self, img, angle, speed):

      if angle <= MAX_ANGLE:
        angle_bucketized = (sum(BINS<np.abs(angle))-1) * np.round(angle/(np.abs(angle)+1e-8)) # 11 -> 1, 5 -> 0, -5 -> 0, -11 -> -1, 
        yield(img, angle_bucketized)

    def bucketize_angle_one_hot(self, img, angle, speed):
      
      if angle <= MAX_ANGLE:

        new_MAX_ANGLE = 2 * MAX_ANGLE
        BINS_2 = np.arange(0, new_MAX_ANGLE - (new_MAX_ANGLE % BUCKETS) + 2*BUCKETS, BUCKETS)
        vector_lenght = int(new_MAX_ANGLE/BUCKETS)
        degree_bucketized = np.zeros((vector_lenght,))

        pos_angle = angle + MAX_ANGLE # Convert all angles to positive
        idx = (sum(BINS_2<np.abs(pos_angle))-1)

        degree_bucketized[idx] = 1
        yield(img, degree_bucketized)

    def normal(self, img, angle, speed):

      if angle <= MAX_ANGLE:
        yield(img, {'angle': angle, 'speed': speed})

    def only_angle(self, img, angle, speed):

      if angle <= MAX_ANGLE:
        yield(img, angle) # /MAX_ANGLE

    def __call__(self, tfRecord):

      dataset = tf.data.TFRecordDataset(tfRecord)
      dataset = dataset.map(generator._parse_features, num_parallel_calls=tf.data.AUTOTUNE)
      dataset = dataset.skip(tf.random.uniform(shape=(), minval=0, maxval=shift_amount, dtype=tf.int64))
      dataset = dataset.window(timesteps, shift=shift_amount, drop_remainder=True)
      dataset = dataset.map(generator.get_inteval)

      new_MAX_ANGLE = 2 * MAX_ANGLE
      BINS_2 = np.arange(0, new_MAX_ANGLE - (new_MAX_ANGLE % BUCKETS), BUCKETS)
      vector_lenght = int(new_MAX_ANGLE/BUCKETS)

      for img, (angle, speed) in dataset:
        if angle <= MAX_ANGLE:
          degree_bucketized = np.zeros((vector_lenght,))
          pos_angle = angle + MAX_ANGLE # Convert all angles to positive
          idx = (sum(BINS_2<=np.abs(pos_angle))-1)
          degree_bucketized[idx] = 1
          yield(img, {'angle': degree_bucketized, 'speed': speed})

          # if idx >= error_margin and idx <= vector_lenght-1 - error_margin:
          #   degree_bucketized[idx - error_margin:idx] = 1
          #   degree_bucketized[idx: idx + 1 + error_margin] = 1
          #   yield(img, {'angle': degree_bucketized, 'speed': speed})

          # if idx < error_margin:
          #   degree_bucketized[:idx] = 1
          #   degree_bucketized[idx: idx + 1 + error_margin] = 1
          #   yield(img, {'angle': degree_bucketized, 'speed': speed})

          # if idx > vector_lenght-1 - error_margin:
          #   degree_bucketized[idx - error_margin:idx] = 1
          #   degree_bucketized[idx:] = 1
            
          #   yield(img, {'angle': degree_bucketized, 'speed': speed})

In [ ]:
method = 'bucketize_angle'
if method == 'normal': output_signature = (tf.TensorSpec(shape=(number_outputs,) + img_shape, dtype=tf.uint8, name=None), {'angle': tf.TensorSpec(shape=(), dtype=tf.float64, name=None), 'speed': tf.TensorSpec(shape=(), dtype=tf.float64, name=None)})
if method == 'bucketize': output_signature = (tf.TensorSpec(shape=(number_outputs,) + img_shape, dtype=tf.uint8, name=None), {'sign': tf.TensorSpec(shape=(), dtype=tf.float64, name=None), 'magnitude': tf.TensorSpec(shape=(90,), dtype=tf.float64, name=None), 'speed': tf.TensorSpec(shape=(), dtype=tf.float64, name=None)})
if method == 'only_angle': output_signature = (tf.TensorSpec(shape=(number_outputs,) + img_shape, dtype=tf.uint8, name=None), tf.TensorSpec(shape=(), dtype=tf.float64, name=None))
if method == 'bucketize_angle': output_signature = (tf.TensorSpec(shape=(number_outputs,) + img_shape, dtype=tf.uint8, name=None), tf.TensorSpec(shape=(), dtype=tf.float64, name=None))
if method == 'one_hot_bucket_angle': output_signature = (tf.TensorSpec(shape=(number_outputs,) + img_shape, dtype=tf.uint8, name=None), tf.TensorSpec(shape=(90,), dtype=tf.float64, name=None))

In [ ]:
ds_train = tf.data.Dataset.from_tensor_slices(train_new_files)
ds_train = ds_train.interleave(lambda filename: tf.data.Dataset.from_generator(generator(method=method),
                                                                               output_signature=output_signature,
                                                                               args=[filename]),
                               cycle_length=30, 
                               num_parallel_calls = AUTOTUNE).cache()

ds_train = ds_train.shuffle(20000)
ds_train = ds_train.batch(BATCH_SIZE, drop_remainder=True)
ds_train = ds_train.prefetch(AUTOTUNE)

In [ ]:
ds_val = tf.data.Dataset.from_tensor_slices(val_new_files)
ds_val = ds_val.interleave(lambda filename: tf.data.Dataset.from_generator(generator(method=method),
                                                                           output_signature=output_signature,
                                                                           args=[filename]),
                           cycle_length=20, 
                           num_parallel_calls = AUTOTUNE).cache()

ds_val = ds_val.shuffle(20000)
ds_val = ds_val.batch(BATCH_SIZE, drop_remainder=True)
ds_val = ds_val.prefetch(AUTOTUNE)

In [ ]:
ds_test = tf.data.Dataset.from_tensor_slices(test_new_files)
ds_test = ds_test.interleave(lambda filename: tf.data.Dataset.from_generator(generator(method=method),
                                                                   output_signature=output_signature,
                                                                   args=[filename]),
                   cycle_length=30, 
                   num_parallel_calls = AUTOTUNE)

ds_test = ds_test.batch(BATCH_SIZE)
ds_test = ds_test.prefetch(AUTOTUNE)

# Modeling

In [ ]:
# Import Model
# Compile with optimizer, loss and metrics
# Initiate Callbacks
# .fit(...)